In [1]:
%load_ext tensorboard
import datetime

import tensorflow as tf
import numpy as np

import dataset
import gpu_utils
gpu_utils.setup_gpus()

BUFFER_SIZE = 10000
BATCH_SIZE = 512  # 512

data = dataset.load_bible(BATCH_SIZE, validation=0.2)

1 Physical GPUs, 1 Logical GPUs


In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(256, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        input_shape=(data.maxlen, data.letters_size),
        batch_size=BATCH_SIZE
    ),
    tf.keras.layers.GRU(data.niqqud_size, return_sequences=True, recurrent_initializer='glorot_uniform'),
])

# best: mean_squared_logarithmic_error, stateless

model.compile(loss='mean_squared_logarithmic_error',
              optimizer='adam',
              metrics=['accuracy'])
model.build()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (512, 236, 512)           446976    
_________________________________________________________________
gru_1 (GRU)                  (512, 236, 14)            22176     
Total params: 469,152
Trainable params: 469,152
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.load_weights(tf.train.latest_checkpoint('niqqud_checkpoints/'))

In [4]:
%tensorboard --logdir logs\fit

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 6660.

In [4]:
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(data.input_texts, data.niqqud_texts,
          batch_size=BATCH_SIZE,
          epochs=10,
          validation_data=(data.input_validation, data.niqqud_validation),
          callbacks=[
              # tf.keras.callbacks.ModelCheckpoint(filepath='niqqud_checkpoints/ckpt_{epoch}', save_weights_only=True),
              tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1),
              tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=0, min_lr=0.001),
              tensorboard_callback,
          ]
          )

Train on 16384 samples, validate on 4096 samples
Epoch 1/10
16384/16384 [==============================] - 7s 447us/sample - loss: 0.0035 - accuracy: 0.9267 - val_loss: 0.0035 - val_accuracy: 0.9266
Epoch 2/10
16384/16384 [==============================] - 7s 435us/sample - loss: 0.0035 - accuracy: 0.9281 - val_loss: 0.0035 - val_accuracy: 0.9283
Epoch 3/10
16384/16384 [==============================] - 7s 439us/sample - loss: 0.0034 - accuracy: 0.9296 - val_loss: 0.0034 - val_accuracy: 0.9294
Epoch 4/10
16384/16384 [==============================] - 7s 436us/sample - loss: 0.0034 - accuracy: 0.9309 - val_loss: 0.0034 - val_accuracy: 0.9301
Epoch 5/10
16384/16384 [==============================] - 7s 441us/sample - loss: 0.0033 - accuracy: 0.9322 - val_loss: 0.0033 - val_accuracy: 0.9318
Epoch 6/10
16384/16384 [==============================] - 7s 438us/sample - loss: 0.0033 - accuracy: 0.9333 - val_loss: 0.0033 - val_accuracy: 0.9333
Epoch 7/10
16384/16384 [===========================

In [5]:
model.add(tf.keras.layers.Softmax())

def print_predictions(batch):
    results = data.merge(batch, model.predict(batch))

    for r in results:
        print(r)


print_predictions(data.input_texts[0:BATCH_SIZE])

אֶם-יִצְאתִי הַשְדָה וְהָנָה חַלְלִי-חָרָב וְאֶם בְאתִי הַעִיר וְהָנָה תַחַלוֹאִי רְעַב כִי-גִם-נָבִיא גִם-כָהֶן סְחָרו אֶל-אָרֶץ וְלֹא יִדָעו
וְקְרְאתֶם בְעָצָם הַיוֹם הַזָה מִקְרָא-קָדַש יְהַיה לְכֶם כֶל-מִלְאכֶת עַבְדָה לֹא תַעָשו חָקֶת עוֹלֶם בְכֶל-מוֹשְבְתִיכֶם לְדְרְתִיכָם
רְאונִי נְעָרִים וְנַחַבָאו וַיִשִישִים קָמו עַמָדו
אָלָה בְנִי רְחַל אֲשֶר יְלֶד לִיעַקָב כֶל-נָפַש אֲרְבְעָה עָשָר
וְנַתַת אֶל-חָשֶן הַמִשְפַט אֶת-הָאוֹרִים וְאֶת-הַתָמִים וְהָיו עַל-לָב אָהֶרֶן בְבָאו לִפְנֵי יְהוָה וְנָשָא אָהֶרֶן אֶת-מִשְפַט בְנִי-יִשְרָאֵל עַל-לָבו לִפְנֵי יְהוָה תָמִיד
וְאִיש בְמְעוֹן ומַעַשָהו בְכְרָמָל וְהָאִיש גְדוֹל מָאֶד וְלו צְאֶן שְלְשֶת-אַלֹפִים וְאֶלֶף עַזִים וַיְהִי בְגְזֶז אֶת-צְאנו בְכְרָמָל
הַלֶךְ וְקְרְאֶת אֶת-הַדְבְרִים הָאָלָה צְפוֹנָה וְאֲמֶרֶת שובָה מִשְבָה יִשְרָאֵל נְאֶם-יְהוָה לוֹא-אֲפִיל פְנִי בְכֶם כִי-חָסִיד אֲנִי נְאֶם-יְהוָה לֹא אָטוֹר לְעוֹלָם
לִשְמַר אֶת-מְצוֹת יְהוָה וְאֶת-חַקְתִיו אֲשֶר אֲנְכִי מְצוֹךָ הַיוֹם לְטוב לֶךָ
וַיִשְמַעָאֵל בֶן-נַתְנִיה נִמְלַט בְש

In [ ]:
del model